In [1]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import mayavi
import mayavi.mlab

import subprocess
#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


In [63]:
def import_shit():
    import local_models.local_models
    import local_models.algorithms
    import local_models.utils
    import local_models.linear_projections
    import local_models.loggin
    import local_models.TLS_models
    import numpy as np
    import logging
    import ml_battery.log

    logger = logging.getLogger(__name__)

    #reload(local_models.local_models)
    #reload(lm)
    #reload(local_models.loggin)
    #reload(local_models.TLS_models)
    np.warnings.filterwarnings('ignore')

In [3]:
FRESH=False

In [4]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [5]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [6]:
data = np.loadtxt("../data/cmu_campus/oakland_part3_an_training.xyz_label_conf")[:,:3]

In [66]:
(data.shape[0]*100)**(1/3)

154.57322768137988

In [58]:
KERNEL=local_models.local_models.GaussianKernel
RUN = 3

project_dir = "../data/3dscene_{}_{:03d}".format(kernel_names[KERNEL], RUN)

os.makedirs(project_dir, exist_ok=1)

In [8]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
#nodes = mayavi.mlab.points3d(converged[:,0], converged[:,1], converged[:,2], scale_mode='scalar', color=(1,0,0))
#nodes.mlab_source.dataset.point_data.scalars = np.ones(converged.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.05


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(data, axis=1)
mayavi.mlab.view(azimuth=180, elevation=80, distance=1.5*np.average(data_avg))#, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "data"
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()

In [9]:
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(room)

In [8]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(2))
linear_models.fit(data)

In [9]:
avg_pt_dist = np.average(linear_models.index.query(data, k=2)[0][:,1])

In [10]:
avg_pt_dist

0.13714420516231735

In [52]:
#bandwidths = np.linspace(3,34,32)
bandwidths = np.concatenate((
    np.linspace(2,3,32),
    #np.linspace(3,34,32),
    #np.linspace(35,66,32),
    #np.linspace(67,98,32)
))
bandwidths

array([2.        , 2.03225806, 2.06451613, 2.09677419, 2.12903226,
       2.16129032, 2.19354839, 2.22580645, 2.25806452, 2.29032258,
       2.32258065, 2.35483871, 2.38709677, 2.41935484, 2.4516129 ,
       2.48387097, 2.51612903, 2.5483871 , 2.58064516, 2.61290323,
       2.64516129, 2.67741935, 2.70967742, 2.74193548, 2.77419355,
       2.80645161, 2.83870968, 2.87096774, 2.90322581, 2.93548387,
       2.96774194, 3.        ])

In [53]:
bandwidths = bandwidths**2*0.05 + bandwidths
bandwidths

array([2.2       , 2.23876171, 2.27762747, 2.31659729, 2.35567118,
       2.39484912, 2.43413111, 2.47351717, 2.51300728, 2.55260146,
       2.59229969, 2.63210198, 2.67200832, 2.71201873, 2.75213319,
       2.79235172, 2.8326743 , 2.87310094, 2.91363163, 2.95426639,
       2.9950052 , 3.03584807, 3.07679501, 3.11784599, 3.15900104,
       3.20026015, 3.24162331, 3.28309053, 3.32466181, 3.36633715,
       3.40811655, 3.45      ])

In [59]:
pth = os.path.join(project_dir, "varying_bandwidth")
os.makedirs(pth, exist_ok=1)

In [57]:
def converge(bandwidth):
    import_shit()
    for converged in local_models.algorithms.local_tls_shift_till_convergence(linear_models, data, tol=1e-8,
                                         kernel=KERNEL(bandwidth),
                                         report=False):
        pass
    return converged

if FRESH:
    from joblib import Parallel, delayed
    convergededs = Parallel(n_jobs=32)(delayed(converge)(bandwidth) for bandwidth in bandwidths*avg_pt_dist)

    for i, bandwidth in enumerate(bandwidths):
        np.savetxt(os.path.join(pth, "converged{:08.02f}.dat".format(bandwidth)), convergededs[i])
else:
    convergededs = []
    for i, bandwidth in enumerate(bandwidths):
        convergededs.append(np.loadtxt(os.path.join(pth, "converged{:08.02f}.dat".format(bandwidth))))


In [ ]:
def mean_center(data, weights=None):
    return data - np.average(data, axis=0,weights=weights)

In [60]:
for i, bandwidth in enumerate(bandwidths):
    converged = convergededs[i]
    data_avg = np.average(converged, axis=0)
    data_std = np.std(converged, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    #mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
    nodes = mayavi.mlab.points3d(converged[:,0], converged[:,1], converged[:,2], 
                                 scale_mode='scalar', colormap='gist_earth')
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((converged.shape[0],3))*(np.average(data_std)/400)
    nodes.mlab_source.dataset.point_data.scalars = (converged[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    #bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
    #bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


    #mayavi.mlab.axes()

    #mayavi.mlab.view(views[0][1],views[0][0])
    mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
    mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0])
    title = "converged_data_b{:08.02f}".format(bandwidth)
    mayavi.mlab.savefig(os.path.join(pth, "{}.png".format(title)))
    mayavi.mlab.close(figure)

In [40]:
mayavi.mlab.close(all=True)

In [ ]:
range_pct = 0.2
grid_steps = 20
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid_limits = tuple(map(lambda i: slice(graph_bounds[0,i], graph_bounds[1,i], grid_steps*1j), range(graph_bounds.shape[1])))
grid = np.mgrid[grid_limits]
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [ ]:
for converged2 in local_models.algorithms.local_tls_shift_till_convergence(linear_models, grid, tol=1e-10,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [ ]:
converged2

In [ ]:
cloud = pcl.PointCloud()
cloud.from_array(converged2.astype('float32'))
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [ ]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
nodes = mayavi.mlab.points3d(converged2[:,0], converged2[:,1], converged2[:,2], scale_mode='scalar', color=(1,0,0))
nodes.mlab_source.dataset.point_data.scalars = np.ones(converged2.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(converged2, axis=1)
mayavi.mlab.view(azimuth=0, elevation=30, distance=1, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "converged_g000{:05d}".format(grid_steps)
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()